### This ipynb notebook can be used to create questions for different topics  at a time

In [ ]:
import os
from dotenv import load_dotenv
 
load_dotenv()

TOGETHER_KEY = os.getenv('TOGETHER_API_KEY')
OPENAI_KEY = os.getenv('OPENAI_API_KEY')

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langchain_together import ChatTogether
 
from typing import List
 
topic_identifier_system = """Analyze user input and identify the physics topic mentioned in the input. Do not return the input text.
For example:
- "Create questions about velocity" -> "velocity"
- "Explain displacement" -> "displacement"
 
Return only the identified physics topic name as given in the input."""
 
topic_check = ChatPromptTemplate.from_messages([
   ("system", topic_identifier_system),
   ("placeholder", "{messages}")
]) | ChatTogether(
    model="meta-llama/Llama-3.3-70B-Instruct-Turbo", temperature=0
    )
 
input = "Create 5 questions on energy."
 
topic = topic_check.invoke({"messages": [("user", input)]}).content

In [ ]:
import pandas as pd

from langchain_core.prompts import ChatPromptTemplate
from langchain_together import ChatTogether

import json

system_prompt = (
    "Create one multiple choice question for {skill} level of Bloom's taxonomy for a 12th grade Physics student in India on {topic}."
    "Requirements:"
    "Student should only be able to answer if they've mastered the concept"
    "Each distractor must address either: A specific misconception about {topic} or A prerequisite knowledge gap"
    "Language and complexity suitable for 12th grade"
    "Physics context and application"
    "Format as the output as a JSON:"
 
        """{{{{
        "question": "",
        "skill": ""
        "options": {{"a": "", "b": "", "c": "", "d": ""}},
        "correct": "",
        "explanation": {{
            "correct": "",
            "a": "misconception/prerequisite tested",
            "b": "", "c": "", "d": ""
        }}
        }}}}"""
 
    "For {skill} level, ensure:"
    "{skill_requirement}"
    "Make sure there are no additional information being other than the output in the format that is asked for."
    "Since you are not capable of creating images, ensure the question does not reference any image. The question should be fully self-contained"
)

skill_requirements = {
   "Remember": "Question tests ability to retrieve relevant knowledge from long-term memory.",
   "Understand": "Question tests ability to onstruct meaning from instructional messages, including oral, written, and graphic communication.",
   "Apply": "Question tests ability to carry out or use a procedure in a given situation.",
   "Analyze": "Question tests ability to break material into foundational parts and determine how parts relate to one another and the overall structure or purpose.",
   "Evaluate": "Question tests ability to make judgments based on criteria and standards."
}
 
skills = ["Remember", "Understand", "Apply", "Analyze", "Evaluate"]

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

llm = ChatTogether(
    model="meta-llama/Llama-3.3-70B-Instruct-Turbo"
    )

query = {"input": '{input}'}

topics = [ ] # Add topics here


def generate_assessment(llm):
    all_responses = []
    model_name = llm.model_name.split('/')[-1]
    
    # Process all topics
    for topic in topics:
        responses = {
            "topic": topic,
            "questions": []
        }
        
        for skill in skills:
            prompt = system_prompt.format(
                skill=skill,
                topic=topic,
                skill_requirement=skill_requirements[skill]
            )
            response = llm.invoke(prompt)
            cleaned_content = response.content.strip()
            if cleaned_content.startswith("```json"):
                cleaned_content = cleaned_content[7:-3]
                
            try:
                question_json = json.loads(cleaned_content)
                responses["questions"].append(question_json)
            except json.JSONDecodeError as e:
                print(f"Error parsing {topic} - {skill} response")
                print(response)
        
        # Save individual JSON if needed
        json_filename = f"LLM_{topic}_{model_name}.json"
        with open(json_filename, "w") as f:
            json.dump(responses, f, indent=2, ensure_ascii=False)
            
        all_responses.extend(
            {"topic": topic, **question} 
            for question in responses["questions"]
        )
    
    # Convert all responses to CSV
    csv_data = []
    max_options = 0
    
    # First pass to find maximum number of options across all questions
    for response in all_responses:
        max_options = max(max_options, len(response.get("options", [])))
    
    # Second pass to create rows
    for response in all_responses:
        options = response.get("options", {})
        explanations = response.get("explanation", {})
        
        row = {
            "topic": response["topic"],
            "skill": response.get("skill", ""),
            "question": response.get("question", ""),
            "correct_option": response.get("correct", ""),
            "correct_explanation": explanations.get("correct", "")
        }
        
        # Add option and explanation columns vrgg for each letter
        for letter in ['a', 'b', 'c', 'd']:
            row[f"option_{letter}"] = options.get(letter, "")
            row[f"explanation_{letter}"] = explanations.get(letter, "")
        
        csv_data.append(row)
    
    # Save consolidated CSV
    csv_filename = f"LLM_all_topics_{model_name}.csv"
    if csv_data:
        df = pd.DataFrame(csv_data)
        df.to_csv(csv_filename, index=False)
        

generate_assessment(llm)